In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

In [2]:
def get_df(link):
  path = 'https://drive.google.com/uc?export=download&id='+link.split('/')[-2]
  return pd.read_csv(path)

In [3]:
url_train = 'https://drive.google.com/file/d/14C3fWUQk9fuv_BaFFN87ZckAPDDsuFd2/view'
url_test = 'https://drive.google.com/file/d/1EULSxKCsdmY6XLBbYzXsrX2FBzx1cdCB/view'

In [4]:
df_train = get_df(url_train)
df_train.head()

,Unnamed: 0,Год,Месяц,День,Время,Канал,Все 18+_TVR,Все 55+_TVR,Все 18+_Share,Все 55+_Share,Дата_День,next_day,Дата
0,0,2018,Ноябрь,Day 1,05:00:00 - 06:00:00,ПЕРВЫЙ КАНАЛ,0.21,0.12,5.23,2.00,2018-11-01 00:00:00,False,2018-11-01 05:00:00
1,1,2018,Ноябрь,Day 1,06:00:00 - 07:00:00,ПЕРВЫЙ КАНАЛ,0.71,0.33,11.23,4.90,2018-11-01 00:00:00,False,2018-11-01 06:00:00
2,2,2018,Ноябрь,Day 1,07:00:00 - 08:00:00,ПЕРВЫЙ КАНАЛ,1.10,0.59,14.58,6.49,2018-11-01 00:00:00,False,2018-11-01 07:00:00
3,3,2018,Ноябрь,Day 1,08:00:00 - 09:00:00,ПЕРВЫЙ КАНАЛ,0.96,0.94,10.41,7.41,2018-11-01 00:00:00,False,2018-11-01 08:00:00
4,4,2018,Ноябрь,Day 1,09:00:00 - 10:00:00,ПЕРВЫЙ КАНАЛ,1.94,3.81,17.54,21.51,2018-11-01 00:00:00,False,2018-11-01 09:00:00


In [5]:
df_test = get_df(url_test)
df_test.head()

,Unnamed: 0,Год,Месяц,День,Время,Канал,Все 18+_TVR,Все 55+_TVR,Все 18+_Share,Все 55+_Share,Дата_День,next_day,Дата
0,19,2018,Ноябрь,Day 1,24:00:00 - 25:00:00,ПЕРВЫЙ КАНАЛ,NaN,NaN,NaN,NaN,2018-11-02 00:00:00,True,2018-11-02 00:00:00
1,20,2018,Ноябрь,Day 1,25:00:00 - 26:00:00,ПЕРВЫЙ КАНАЛ,NaN,NaN,NaN,NaN,2018-11-02 00:00:00,True,2018-11-02 01:00:00
2,21,2018,Ноябрь,Day 1,26:00:00 - 27:00:00,ПЕРВЫЙ КАНАЛ,NaN,NaN,NaN,NaN,2018-11-02 00:00:00,True,2018-11-02 02:00:00
3,22,2018,Ноябрь,Day 1,27:00:00 - 28:00:00,ПЕРВЫЙ КАНАЛ,NaN,NaN,NaN,NaN,2018-11-02 00:00:00,True,2018-11-02 03:00:00
4,23,2018,Ноябрь,Day 1,28:00:00 - 29:00:00,ПЕРВЫЙ КАНАЛ,NaN,NaN,NaN,NaN,2018-11-02 00:00:00,True,2018-11-02 04:00:00


In [6]:
df_train.shape

(70080, 13)

In [7]:
df_test.shape

(17640, 13)

In [8]:
df_train[df_train['День'] != 'Day 1'][:10]

,Unnamed: 0,Год,Месяц,День,Время,Канал,Все 18+_TVR,Все 55+_TVR,Все 18+_Share,Все 55+_Share,Дата_День,next_day,Дата
95,139,2018,Ноябрь,Day 2,24:00:00 - 25:00:00,ПЕРВЫЙ КАНАЛ,2.11,2.99,8.52,8.63,2018-11-03 00:00:00,True,2018-11-03 00:00:00
96,140,2018,Ноябрь,Day 2,25:00:00 - 26:00:00,ПЕРВЫЙ КАНАЛ,0.85,0.61,5.60,3.05,2018-11-03 00:00:00,True,2018-11-03 01:00:00
97,141,2018,Ноябрь,Day 2,26:00:00 - 27:00:00,ПЕРВЫЙ КАНАЛ,0.18,0.06,1.92,0.47,2018-11-03 00:00:00,True,2018-11-03 02:00:00
98,142,2018,Ноябрь,Day 2,27:00:00 - 28:00:00,ПЕРВЫЙ КАНАЛ,0.14,0.00,1.97,0.00,2018-11-03 00:00:00,True,2018-11-03 03:00:00
99,143,2018,Ноябрь,Day 2,28:00:00 - 29:00:00,ПЕРВЫЙ КАНАЛ,0.30,0.00,5.74,0.00,2018-11-03 00:00:00,True,2018-11-03 04:00:00
100,163,2018,Ноябрь,Day 2,24:00:00 - 25:00:00,РОССИЯ 1,4.04,8.05,16.28,23.25,2018-11-03 00:00:00,True,2018-11-03 00:00:00
101,164,2018,Ноябрь,Day 2,25:00:00 - 26:00:00,РОССИЯ 1,1.82,3.56,11.91,17.94,2018-11-03 00:00:00,True,2018-11-03 01:00:00
102,165,2018,Ноябрь,Day 2,26:00:00 - 27:00:00,РОССИЯ 1,0.71,2.00,7.54,15.81,2018-11-03 00:00:00,True,2018-11-03 02:00:00
103,166,2018,Ноябрь,Day 2,27:00:00 - 28:00:00,РОССИЯ 1,0.56,1.57,7.71,16.54,2018-11-03 00:00:00,True,2018-11-03 03:00:00
104,167,2018,Ноябрь,Day 2,28:00:00 - 29:00:00,РОССИЯ 1,0.36,1.01,6.80,16.27,2018-11-03 00:00:00,True,2018-11-03 04:00:00


In [13]:
df_train["Канал"].value_counts().index

Index(['СТС', 'ПЕРВЫЙ КАНАЛ', 'РЕН ТВ', 'РОССИЯ 1', 'ТНТ'], dtype='object')

In [10]:
df_train.drop(columns= 'Unnamed: 0', inplace=True)

In [11]:
df_train.groupby(['Год', 'Месяц', 'День']).mean()

Все 18+_TVR  Все 55+_TVR  ...  Все 55+_Share  next_day
Год  Месяц   День                              ...                         
2018 Декабрь Day 1      1.535053     2.422632  ...       6.797684  0.000000
             Day 10     1.413250     2.164583  ...       8.172500  0.208333
             Day 11     1.324917     2.182417  ...       7.394250  0.208333
             Day 12     1.364333     2.125083  ...       7.518167  0.208333
             Day 13     1.447583     2.219833  ...       8.119417  0.208333
...                          ...          ...  ...            ...       ...
2020 Январь  Day 5      0.885200     1.302000  ...       8.016400  1.000000
             Day 6      1.266083     1.855917  ...       6.452000  0.208333
             Day 7      1.237750     1.613417  ...       5.322417  0.208333
             Day 8      1.531500     1.910833  ...       6.139667  0.208333
             Day 9      1.181583     1.702417  ...       7.261500  0.208333

[712 rows x 5 columns]

In [ ]:
#посмотреть средней для кажого канала по месяцу, дню недели

In [18]:
month_mean = {chanel : df_train.groupby(['Год', 'Месяц']).mean() for chanel in df_train["Канал"].value_counts().index}

In [ ]:
month_mean['ПЕРВЫЙ КАНАЛ']

In [22]:
def code_year(year):
  return (6 + year % 100 + (year % 100) // 4) % 7 

In [21]:
def code_month(month):
  if month == 'Январь' or month == 'Октябрь':
    return 1
  elif month == 'Май':
    return 2
  elif month == 'Август':
    return 3
  elif month == 'Февраль' or month == 'Март' or month == 'Ноябрь':
    return 4
  elif month == 'Июнь':
    return 5
  elif month == 'Декабрь' or month == 'Сентябрь':
    return 6
  else:
    return 0

In [23]:
def code_day(day):
  return int(day.split()[1])

In [26]:
len(df_train)

70080

In [30]:
df_train['День недели'] = pd.Series([(code_day(df_train['День'][i]) + code_month(df_train['Месяц'][i]) + code_year(df_train['Год'][i])) % 7 for i in range(len(df_train))])

In [37]:
day_of_week_chanel = {chanel : df_train.groupby(['Год', 'День недели']).mean() for chanel in df_train['Канал'].value_counts().index}

In [38]:
day_of_week_chanel['СТС']

Все 18+_TVR  Все 55+_TVR  ...  Все 55+_Share  next_day
Год  День недели                            ...                         
2018 0               1.430762     2.135833  ...       7.212857  0.208333
     1               1.561286     2.261190  ...       7.036762  0.208333
     2               1.531787     2.268831  ...       8.373843  0.252809
     3               1.387519     2.215112  ...       8.073594  0.187166
     4               1.401288     2.182049  ...       7.943178  0.184049
     5               1.420092     2.255373  ...       8.236601  0.130719
     6               1.291586     1.943910  ...       7.402486  0.315315
2019 0               1.151793     1.593629  ...       6.221368  0.204387
     1               1.298372     1.781628  ...       6.421457  0.208333
     2               1.222735     1.874899  ...       7.851920  0.220220
     3               1.293137     1.985154  ...       7.868375  0.181818
     4               1.220009     1.855495  ...       7.661545  0.234205
     5               1.238488     1.881985  ...       7.693533  0.212150
     6               1.241486     1.850042  ...       7.502329  0.200765
2020 0               1.171477     1.730040  ...       6.966197  0.208333
     1               1.255655     1.768005  ...       6.794879  0.232558
     2               1.209866     1.809112  ...       7.464097  0.203851
     3               1.214658     1.844635  ...       7.744810  0.199063
     4               1.183017     1.820968  ...       7.652527  0.208333
     5               1.216135     1.875148  ...       7.706059  0.213018
     6               1.227578     1.904384  ...       7.842796  0.194346

[21 rows x 5 columns]